In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Load the "noharshi" dataset
data = pd.read_csv('/content/nosravya.csv')

# Convert the 'datetime' column to a datetime object
data['datetime'] = pd.to_datetime(data['datetime'])

# Set 'datetime' column as the index
data.set_index('datetime', inplace=True)

# Define the target date for training
target_date = pd.to_datetime('2023-08-20')
def create_sequences(data, sequence_length):
    sequences = []
    for i in range(len(data) - sequence_length):
        seq = data[i:i+sequence_length]
        label = data[i+sequence_length]
        sequences.append((seq, label))
    return sequences

# List of column names to predict
columns_to_predict = ['temp', 'humidity', 'precip']  # Add more columns if needed

# Initialize dictionaries to store forecasted values and summary statistics
forecasted_weather_data = {}
summary_statistics = {}

# Iterate through each column
for column_name in columns_to_predict:
    print(f"Predicting {column_name}...")

    data_target = data[column_name]

    # Split the data into training and testing sets
    train_data = data_target.iloc[:-180]
    test_data = data_target.iloc[-180:]

    # Normalize the data
    scaler = MinMaxScaler()
    train_data_normalized = scaler.fit_transform(np.array(train_data).reshape(-1, 1))

    # Prepare data for LSTM
    sequence_length = 60
    train_sequences = create_sequences(train_data_normalized, sequence_length)

    X_train = np.array([seq for seq, label in train_sequences])
    y_train = np.array([label for seq, label in train_sequences])

    # Build the LSTM model
    model = Sequential([
        LSTM(50, activation='relu', input_shape=(sequence_length, 1)),
        Dense(1)
    ])

    model.compile(optimizer='adam', loss='mean_squared_error')

    # Train the model
    model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1)

    # Prepare data for predicting the next 60 days
    last_60_days = data_target.iloc[-60:]
    last_60_days_normalized = scaler.transform(np.array(last_60_days).reshape(-1, 1))

    # Create a single sequence for the last 60 days
    input_sequence = last_60_days_normalized.reshape(1, -1, 1)

    # Make predictions for the next 60 days
    next_60_days_predictions = []
    for _ in range(60):
        prediction = model.predict(input_sequence)
        next_60_days_predictions.append(prediction[0, 0])  # Get the prediction value
        input_sequence = np.roll(input_sequence, -1, axis=1)  # Shift values one step
        input_sequence[0, -1, 0] = prediction  # Update the last value with the prediction

    # Inverse normalize the predictions
    next_60_days_predictions = scaler.inverse_transform(np.array(next_60_days_predictions).reshape(-1, 1))

    # Prepare index for the next 60 days
    next_60_days_index = pd.date_range(start=target_date + pd.DateOffset(days=1), periods=60)

    # Store forecasted values for the current feature
    forecasted_weather_data[column_name] = next_60_days_predictions

    # Calculate summary statistics for the forecasted data
    mean_forecast = np.mean(next_60_days_predictions)
    min_forecast = np.min(next_60_days_predictions)
    max_forecast = np.max(next_60_days_predictions)
    summary_statistics[column_name] = {
        'Mean Forecast': mean_forecast,
        'Min Forecast': min_forecast,
        'Max Forecast': max_forecast
    }




Predicting temp...
Epoch 1/50
210/210 [==============================] - 9s 34ms/step - loss: 0.0190
Epoch 2/50
210/210 [==============================] - 5s 22ms/step - loss: 0.0038
Epoch 3/50
210/210 [==============================] - 5s 23ms/step - loss: 0.0034
Epoch 4/50
210/210 [==============================] - 7s 34ms/step - loss: 0.0031
Epoch 5/50
210/210 [==============================] - 5s 23ms/step - loss: 0.0030
Epoch 6/50
210/210 [==============================] - 5s 23ms/step - loss: 0.0028
Epoch 7/50
210/210 [==============================] - 7s 34ms/step - loss: 0.0027
Epoch 8/50
210/210 [==============================] - 5s 22ms/step - loss: 0.0024
Epoch 9/50
210/210 [==============================] - 7s 32ms/step - loss: 0.0026
Epoch 10/50
210/210 [==============================] - 9s 42ms/step - loss: 0.0023
Epoch 11/50
210/210 [==============================] - 5s 24ms/step - loss: 0.0024
Epoch 12/50
210/210 [==============================] - 6s 28ms/step - loss: 0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report


# Calculate similarity scores and recommend crops based on the forecasted weather features
future_weather_data = {
    'temperature': summary_statistics['temp']['Mean Forecast'],
    'humidity': summary_statistics['humidity']['Mean Forecast'],
    'rainfall': summary_statistics['precip']['Mean Forecast']
}

crop_data_ = pd.read_csv('/content/finalcrop data set.csv')

# Shuffle the dataset
crop_data = crop_data_.sample(frac=1, random_state=42)  # frac=1 shuffles the entire dataset

# Select features (N, P, K, pH, temperature, humidity, rainfall, soil type)
X = crop_data.drop(columns=['label'])
y = crop_data['label']  # Crop labels (the target variable)

# Encode categorical variables (soiltype) using one-hot encoding
X_encoded = pd.get_dummies(X, columns=['soiltype'])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Initialize and train a decision tree classifier
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train, y_train)

# Evaluate the model on the testing data
y_pred = clf.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred))


In [ ]:


input_soil_type = input("Enter your soil type: ")
input_N = float(input("Enter the N value: "))
input_P = float(input("Enter the P value: "))
input_K = float(input("Enter the K value: "))
input_pH = float(input("Enter the pH value: "))

# Initialize a variable to keep track of the highest overall similarity score
highest_similarity_score = 0

# Initialize a variable to store the recommended crop
recommended_crop = None

# Loop through the crops in the dataset
for index, row in crop_data.iterrows():
    # Check if the soil type matches the input soil type
    if row['soiltype'] == input_soil_type:
        crop_weather_data = {
            'temperature': row['temperature'],
            'humidity': row['humidity'],
            'rainfall': row['rainfall']
        }

        # Calculate Euclidean distance between predicted and actual weather data
        weather_distance = np.sqrt(np.sum(np.square(np.array(list(future_weather_data.values())) - np.array(list(crop_weather_data.values())))))

        # Calculate Euclidean distance between input soil parameters and crop soil parameters
        soil_distance = np.sqrt((input_N - row['N'])**2 + (input_P - row['P'])**2 + (input_K - row['K'])**2 + (input_pH - row['ph'])**2)

        # Use the inverse of the distance as the similarity score for weather and soil
        weather_similarity_score = 1 / (1 + weather_distance)
        soil_similarity_score = 1 / (1 + soil_distance)

        # Calculate the overall similarity score as a weighted average
        overall_similarity_score = (weather_similarity_score + soil_similarity_score) / 2

        # Check if the current crop has a higher overall similarity score than the previous highest score
        if overall_similarity_score > highest_similarity_score:
            highest_similarity_score = overall_similarity_score
            recommended_crop = row['label']

# Print the recommended crop
if recommended_crop is not None:
    print(f"Recommended Crop for Soil Type '{input_soil_type}': {recommended_crop} (Overall Similarity Score: {highest_similarity_score:.2f})")
else:
    print(f"No matching crop found for Soil Type '{input_soil_type}'")


In [ ]:
# Save the LSTM model to a file
model.save('lstm_model.h5')
